# An Introduction to Dynamic Mode Decomposition (DMD)

## Goal:

Approximate the leading eigencomposition of (high dimensional) linear operator $A \in \mathbb{C}^{n \times n}$ where $X'=AX$ to find spacial temporal coherant modes of the (possibly non-linear) system. 

## The DMD Ingredients:

$n$ is the number of spatial points saved per shot and $m$ is the number of snapshots taken.

- Snapshot of fluid flow (long and skinny) $X = 
 \begin{bmatrix}
  \vert & \vert &     & \vert \\
  x_1   & x_2   & ... & x_{m-1} \\
  \vert & \vert &     & \vert
 \end{bmatrix} \in \mathbb{C}^{n \times (m-1)}$

- Snapshot of fluid flow evolved by one unit in time $X' = 
 \begin{bmatrix}
  \vert & \vert &     & \vert \\
  x_2   & x_3   & ... & x_{m} \\
  \vert & \vert &     & \vert
 \end{bmatrix} \in \mathbb{C}^{n \times (m-1)}$

In [46]:
import math #since torch doesn't have a built in pi function
import torch
import numpy as np

## The DMD Recipe:

**1. Compute Singular Value Decomposition (SVD) of big data matrix $X$ to find the dominant coherent structures (Columns of $U$).**

$$ X = U\Sigma V^* $$

The exact SVD can be reduced to an economy SVD if, for example, 99% of the system energy is captured by the first $r$ comumns of $U$. The * denotes the conjugate transpose.

**2. Project $A$ on the dominant singular vectors $U^*$ and $U$ to get the reduced dynamic operator $\tilde{A}$**

$$U^*X'V\Sigma^{-1} = U^*AU = \tilde{A}$$

We do not calculate $A$ using $ X' = AU\Sigma V^* $ because this would be too demanding. $\tilde{A}$, which is only of the magnitude of time, is a linear best fit dynamical system, that tell you how your POD modes evolve over time.

**3. Compute the eigenvalues $\Lambda$ and eigenvectors $W$ of $\tilde{A}$**

$$\tilde{A}W = W\Lambda$$

$\tilde{A}$ has the same non-zero *eigenvalues* as $A$.

**4. Compute the eigenvectors $\Phi$ of $A$**

$\Phi = X'V\Sigma^{-1}W$

Also called modes.

## PyTorch:

In [66]:
def svd(matrix, rank=None):
    U, s, V = torch.svd(matrix, some=False)  #V does not need to be conjugate transposed... idk why
#    print('V {}:\n{}'.format(V.shape, V))
#    print('U {}:\n{}'.format(U.shape, U))
    if rank is None:
        rank = s.shape[0]
        print('rank: {}'.format(rank))
    return U[:, :rank], s[:rank], V[:, :rank]


def dmd(matrix, rank=None):
    U, s, V = svd(matrix[:,:-1], rank)
    print('torch.diag(1.0/s):\n {}'.format(torch.diag(1.0/s)))
    print(U)                                                #is conj from numpy?
    At = U.conj().T @ matrix[:, 1:] @ V @ torch.diag(1.0/s) #idk if U.conj().T right, cause print weird. U, matrix, V, diag are right
    val, vec = torch.eig(At)
    phi = matrix[:, 1:] @ V @ torch.diagonal(1.0/s) @ vec
    return val, phi


def f1(Xm, Tm, z):
    return torch.multiply(20-0.2*torch.pow(Xm, 2), torch.exp(z*Tm)).T


def f2(Xm, Tm, z):
    return torch.multiply(Xm, torch.exp(z*Tm)).T


def f3(Xm, Tm, z):
    return torch.multiply(10*torch.tanh(Xm/2)/torch.cosh(Xm/2), torch.exp(z*Tm)).T


def create_mesh(x_start, x_end, n_x, t_start, t_end, n_t):
    x = torch.linspace(x_start, x_end, n_x)
    t = torch.linspace(t_start, t_end, n_t)
    return torch.meshgrid(t, x)


def main():
    Tm, Xm = create_mesh(-10, 10, 100, 0, 6*math.pi, 80)
    X_1r = f1(Xm, Tm, -0.05+2.3j)
    X_2 = f2(Xm, Tm, 0.6j)
    X_3r = f3(Xm, Tm, 0.1+2.8j)
    data = X_1r + X_2 + X_3r

    rank=3
    dmd(data, rank=rank)

if __name__ == "__main__":
    main()

torch.diag(1.0/s):
 tensor([[0.0011, 0.0000, 0.0000],
        [0.0000, 0.0013, 0.0000],
        [0.0000, 0.0000, 0.0022]])
tensor([[-7.2024e-03-0.0115j, -2.5045e-02+0.0417j, -1.7042e-01-0.0494j],
        [-2.3661e-03-0.0125j, -2.2842e-02+0.0428j, -1.6649e-01-0.0486j],
        [ 2.3692e-03-0.0135j, -2.0709e-02+0.0440j, -1.6254e-01-0.0478j],
        [ 7.0021e-03-0.0146j, -1.8647e-02+0.0452j, -1.5854e-01-0.0469j],
        [ 1.1533e-02-0.0156j, -1.6661e-02+0.0464j, -1.5450e-01-0.0461j],
        [ 1.5960e-02-0.0167j, -1.4754e-02+0.0476j, -1.5042e-01-0.0452j],
        [ 2.0284e-02-0.0177j, -1.2931e-02+0.0489j, -1.4628e-01-0.0443j],
        [ 2.4503e-02-0.0188j, -1.1195e-02+0.0501j, -1.4208e-01-0.0433j],
        [ 2.8617e-02-0.0200j, -9.5513e-03+0.0514j, -1.3782e-01-0.0424j],
        [ 3.2624e-02-0.0211j, -8.0060e-03+0.0527j, -1.3349e-01-0.0414j],
        [ 3.6525e-02-0.0223j, -6.5646e-03+0.0540j, -1.2907e-01-0.0403j],
        [ 4.0317e-02-0.0236j, -5.2336e-03+0.0555j, -1.2457e-01-0.0392j],
 

RuntimeError: expected scalar type ComplexFloat but found Float

## NumPy:

In [67]:
def svd(matrix, rank=None):
    U, s, V = np.linalg.svd(matrix)
    V = V.conj().T
#    print('V.conj().T {}:\n{}'.format(V.shape, V))
#    print('U {}:\n{}'.format(U.shape, U))
    if rank is None:
        rank = s.shape[0]
        print('rank: {}'.format(rank))
    return U[:, :rank], s[:rank], V[:, :rank]

def dmd(matrix, rank=None):
    U, s, V = svd(matrix[:,:-1], rank)
    print('np.diag(1.0/s):\n {}'.format(np.diag(1.0/s)))
    print(U)
    At = U.conj().T @ matrix[:, 1:] @ V @ np.diag(1.0/s)
    val, vec = np.linalg.eig(At)
    phi = matrix[:, 1:] @ V @ np.diag(1.0/s) @ vec
    return val, phi


def create_mesh(x_start, x_end, n_x, t_start, t_end, n_t):
    x = np.linspace(x_start, x_end, n_x)
    t = np.linspace(t_start, t_end, n_t)
    return np.meshgrid(x, t)


def f1(Xm, Tm, z):
    return np.multiply(20-0.2*np.power(Xm, 2), np.exp(z*Tm)).T


def f2(Xm, Tm, z):
    return np.multiply(Xm, np.exp(z*Tm)).T


def f3(Xm, Tm, z):
    return np.multiply(10*np.tanh(Xm/2)/np.cosh(Xm/2), np.exp(z*Tm)).T


def main():
    Xm, Tm = create_mesh(-10, 10, 100, 0, 6*np.pi, 80)
    # function 1
    X_1 = f1(Xm, Tm, 2.3j)
    # function 2
    X_2 = f2(Xm, Tm, 0.6j)
    # function 3
    X_3 = f3(Xm, Tm, 2.8j)
    # funcion 3 with positive real part
    X_3r = f3(Xm, Tm, 0.1+2.8j)
    # function 1 with negative real part
    X_1r = f1(Xm, Tm, -0.05+2.3j)

    # plot data matrix
    data = X_1r + X_2 + X_3r
#    print('data: {}'.format(len(data)))
    rank=3
    val, phi = dmd(data, rank=rank)

    b = np.linalg.pinv(phi) @ data[:, 0]
    psi = np.zeros((len(Tm[:,0]), rank), dtype="complex")
    dt = Tm[1,0] - Tm[0,0]
    omega = np.log(val) / dt
    for i, om in enumerate(omega):
        psi[:,i] = np.exp(om*Tm[:,0]) * b[i]

if __name__ == "__main__":
    main()

np.diag(1.0/s):
 [[0.00110714 0.         0.        ]
 [0.         0.00131741 0.        ]
 [0.         0.         0.00221918]]
[[-7.20252909e-03-0.01148351j -2.50449843e-02+0.04166018j
  -1.70415298e-01-0.0493759j ]
 [-2.36600464e-03-0.01249915j -2.28419853e-02+0.0428445j
  -1.66493554e-01-0.04858339j]
 [ 2.36904706e-03-0.01352149j -2.07082501e-02+0.04402926j
  -1.62537002e-01-0.04777048j]
 [ 7.00213144e-03-0.01455329j -1.86467636e-02+0.04521807j
  -1.58541800e-01-0.04693577j]
 [ 1.15327020e-02-0.01559761j -1.66608242e-02+0.04641492j
  -1.54503700e-01-0.04607773j]
 [ 1.59601550e-02-0.01665784j -1.47540760e-02+0.04762419j
  -1.50418011e-01-0.04519464j]
 [ 2.02838236e-02-0.01773772j -1.29305436e-02+0.04885075j
  -1.46279551e-01-0.04428462j]
 [ 2.45029716e-02-0.01884137j -1.11946699e-02+0.05009996j
  -1.42082602e-01-0.04334559j]
 [ 2.86167869e-02-0.01997334j -9.55135767e-03+0.05137774j
  -1.37820850e-01-0.04237526j]
 [ 3.26243735e-02-0.02113865j -8.00601382e-03+0.05269061j
  -1.33487336e-0

## Problems:

- Solved
    - pytorch doesn't have a pi-function (therefore we need math.pi)
    - torch.linspace keeps only 4 decimal places
    - V does not need to be conjugate transposed... no idea why
    
    
- torch.meshgrid flipped from numpy.meshgrip

- U and V appear to be the same for the first few values after .svd, then different in the last ones. Is this because the system is only rank=3? similar to sigma, we truncate the rest?
    - svd function gives same results


- Why use diag(1.0/s) instead of .inverse?

- is .conj() from numpy?

## Now what?

With our (spacial) modes $\Phi$ and (temporal) eigenvalues $\Lambda$, we can predict what the system will do in the future.

$$\hat{X}(k\Delta t) = \Phi\Lambda^kb_0$$

$\hat{X}$ is a future state prediction of $X$.

$\Lambda^k$ advances one time increment $\Delta t$ with each $k$.

$b_0$ is amplitude of modes. Condition for how much each mode is expressed in the data.

## Questions:

Is the difference between $\hat{X}$ and $X'$ just that $\hat{X}$ is the predicted future state and $X'$ is the actual future state? If yes, would element $x_m$ in $X'$ be used to validate $\hat{X}$?

Permissions of notebook? using sudo chmod u=rwx,g=rwx,o=rwx right now, which probably is too loose

## References

1. Kutz, J. N., Brunton, S. L. 1., Brunton, B. W., & Proctor, J. L. (2016). *Dynamic Mode Decomposition.* Philadelphia, PA, USA: Society for Industrial and Applied Mathematics.
2. Taylor, R. (2016) Dynamic Mode Decomposition in Python. *Pyrunner.* Accessed: 25 January 2021. http://www.pyrunner.com/weblog/2016/07/25/dmd-python/
    